# **AdapterHub** quickstart example for **training**

This is an adaptation of the HuggingFace [sentence classification notebook](https://colab.research.google.com/github/huggingface/blog/blob/master/notebooks/trainer/01_text_classification.ipynb).

First, install adapter-transformers from github/master, download the SST dataset, and import the required modules:

In [ ]:
!pip install git+https://github.com/adapter-hub/adapter-transformers.git

In [ ]:
!git clone https://github.com/huggingface/transformers
!python transformers/utils/download_glue_data.py --tasks SST

In [3]:
import dataclasses
import logging
import os
import sys
from dataclasses import dataclass, field
from typing import Dict, Optional

import numpy as np

import torch
from transformers import AutoTokenizer, EvalPrediction, GlueDataset, GlueDataTrainingArguments, AutoModelWithHeads, AdapterType
from transformers import GlueDataTrainingArguments as DataTrainingArguments
from transformers import (
    Trainer,
    TrainingArguments,
    glue_compute_metrics,
    glue_tasks_num_labels,
    set_seed,
)

Training a new task adapter requires only few modifications compared to fully fine-tuning a model with Hugging Face's Trainer. We first configure the training and data arguments (which we would usually set via the command line):

In [21]:
model_name = "roberta-base"

data_args = GlueDataTrainingArguments(task_name="sst-2", data_dir="./glue_data/SST-2")
training_args = TrainingArguments(
    logging_steps=1000, 
    per_device_train_batch_size=32, 
    per_device_eval_batch_size=64, 
    save_steps=1000,
    evaluate_during_training=True,
    output_dir="./models/sst-2",
    overwrite_output_dir=True,
    do_train=True,
    do_eval=True,
    do_predict=True,
    learning_rate=0.0001,
    num_train_epochs=3,
)
set_seed(training_args.seed)
num_labels = glue_tasks_num_labels[data_args.task_name]

We then load a pre-trained model (*roberta-base*) and add a new *sst-2* task adapter:

In [13]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelWithHeads.from_pretrained(model_name)
model.add_adapter("sst-2", AdapterType.text_task)
model.train_adapter(["sst-2"])

By calling `train_adapter(["sst-2"])` we freeze all transformer parameters except for the parameters of sst-2 adapter. Before training we add a new classification head to our model:

In [14]:
model.add_classification_head("sst-2", num_labels=num_labels)
model.set_active_adapters([["sst-2"]])

The weights of this classification head can be stored together with the adapter weights to allow for a full reproducibility. The method call model.set_active_adapters([["sst-2"]]) registers the sst-2 adapter as a default for training. This also supports adapter stacking and adapter fusion!

We can then train our adapter using the Hugging Face Trainer:

In [ ]:
train_dataset = GlueDataset(data_args, tokenizer=tokenizer)
eval_dataset = GlueDataset(data_args, tokenizer=tokenizer, mode="dev")

def compute_metrics(p: EvalPrediction):
    preds = np.argmax(p.predictions, axis=1)
    return glue_compute_metrics(data_args.task_name, preds, p.label_ids)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()
trainer.evaluate()

That's it! `model.save_all_adapters('output-path')` exports all adapters. Consider sharing your adapters on AdapterHub!

In [ ]:
model.save_all_adapters(".")
!ls -l sst-2